In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from Data_Preparation.Preparation import CustomDataLoader
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.metrics import Recall, Loss
from ignite.contrib.handlers import ProgressBar
from ignite.contrib.handlers.param_scheduler import LRScheduler
import optuna
from ignite.engine import create_supervised_trainer, create_supervised_evaluator

C:\Users\rdede\AppData\Local\Temp\ipykernel_22392\1572454150.py:9: DeprecationWarning: c:\Users\rdede\Documents\GitHub\SnowDetection\.env\lib\site-packages\ignite\contrib\handlers\param_scheduler.py has been moved to /ignite/handlers/param_scheduler.py and will be removed in version 0.6.0.
 Please refer to the documentation for more details.
  from ignite.contrib.handlers.param_scheduler import LRScheduler
c:\Users\rdede\Documents\GitHub\SnowDetection\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# class MyCNNModel(nn.Module):
#     def __init__(self, n_filters, kernel_size):
#         super(MyCNNModel, self).__init__()
#         self.conv1 = nn.Conv2d(3, n_filters, kernel_size=kernel_size)
#         self.conv2 = nn.Conv2d(n_filters, n_filters*2, kernel_size=kernel_size)
#         self.conv3 = nn.Conv2d(n_filters*2, n_filters*4, kernel_size=kernel_size)
#         self.flatten = nn.Flatten()

#     def forward(self, x):
#         x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
#         x = torch.relu(torch.max_pool2d(self.conv2(x), 2))
#         x = torch.relu(torch.max_pool2d(self.conv3(x), 2))
#         x = self.flatten(x)
#         flatten_shape = x.shape[1]
#         self.fc1 = nn.Linear(flatten_shape, 50)  # Define the linear layer with the correct input size
#         x = torch.relu(self.fc1(x))
#         self.fc2 = nn.Linear(50, 2)  # Define the output layer
#         x = self.fc2(x)
#         return x

# criterion = nn.CrossEntropyLoss()
# optimizer_fn = optim.Adam

# def objective(trial):
#     lr = trial.suggest_float('lr', 0.001, 0.1)
#     batch_size = trial.suggest_categorical('batch_size', [4, 8, 16, 32])
#     num_epochs = trial.suggest_categorical('num_epochs', [5, 10, 15,20, 25, 30, 35, 40, 45, 50])
#     n_filters = trial.suggest_categorical('n_filters', [4, 8, 16, 32])
#     kernel_size = trial.suggest_categorical('kernel_size', [3, 5, 7])

#     model = MyCNNModel(n_filters=n_filters, kernel_size=kernel_size)
#     optimizer = optimizer_fn(model.parameters(), lr=lr)

#     train_loader = CustomDataLoader(data_path="./data", batch_size=batch_size, dataset_type="train").data_loader
#     val_loader = CustomDataLoader(data_path="./data", batch_size=batch_size, dataset_type="val").data_loader

#     trainer = create_supervised_trainer(model, optimizer, criterion)
#     train_recall_metric = Recall()
#     train_loss_metric = Loss(criterion)
#     evaluator = create_supervised_evaluator(model, metrics={'recall': train_recall_metric, 'loss': train_loss_metric})

#     @trainer.on(Events.EPOCH_COMPLETED)
#     def compute_train_metrics(engine):
#         train_recall_metric.reset()
#         train_loss_metric.reset()
#         for batch in train_loader:
#             x, y = batch
#             y_pred = model(x)
#             loss = criterion(y_pred, y)
#             train_recall_metric.update((y_pred, y))
#             train_loss_metric.update((y_pred, y))  # Pass y_pred and y to update

#     early_stopping = EarlyStopping(patience=3, score_function=lambda engine: torch.mean(train_recall_metric.compute()).item(), trainer=trainer)
#     checkpoint = ModelCheckpoint(dirname='./checkpoints', filename_prefix='best', score_function=lambda engine: torch.mean(train_recall_metric.compute()).item(), score_name='recall', n_saved=1, require_empty=False)

#     trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint, {'model': model, 'optimizer': optimizer})
#     trainer.add_event_handler(Events.EPOCH_COMPLETED, early_stopping)

#     trainer.run(train_loader, max_epochs=num_epochs)
#     evaluator.run(val_loader)
#     recall = train_recall_metric.compute()
#     print(f'Validation Recall: {torch.mean(recall).item():.4f}')
    
#     return {'recall': torch.mean(recall).item()}

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10)

# best_hyperparams = study.best_params
# best_metric_value = study.best_value

# print(f'Best hyperparameters: {best_hyperparams}')
# print(f'Best metric value: {best_metric_value:.4f}')

In [4]:
from kan import KAN

def create_kan():
    return KAN(width=[128**2, 1, 2], grid=3, k=3)
model = create_kan()

In [4]:
# from kan import KAN

# class MyCNNModel(nn.Module):
#     def __init__(self, kernel_size):
#         super(MyCNNModel, self).__init__()
#         self.conv1 = nn.Conv2d(3, 128, kernel_size=kernel_size)
#         self.conv2 = nn.Conv2d(128, 256, kernel_size=kernel_size)
#         self.conv3 = nn.Conv2d(256, 512, kernel_size=kernel_size)
#         self.flatten = nn.Flatten()
#         self.kan = KAN(width=[73728,20,2], grid=3, k=3, seed=0,device='cuda:0')
    
#     def forward(self,x):
#         x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
#         x = torch.relu(torch.max_pool2d(self.conv2(x), 2))
#         x = torch.relu(torch.max_pool2d(self.conv3(x), 2))
#         x = self.flatten(x)
#         out = self.kan(x)
#         return out



In [5]:
# define training hyperparameters
INIT_LR = 0.01
BATCH_SIZE = 4
EPOCHS = 25
N_FILTERS = 4
KERNEL_SIZE = 5

# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
train_set = CustomDataLoader(data_path="./data", batch_size=BATCH_SIZE, dataset_type="train")
val_set = CustomDataLoader(data_path="./data", batch_size=BATCH_SIZE, dataset_type="val")

In [7]:
# calculate steps per epoch for training and validation set
trainSteps = len(train_set) // BATCH_SIZE
valSteps = len(val_set) // BATCH_SIZE

In [8]:
def test_acc():
    with torch.no_grad():
        predictions = torch.argmax(model(val_set.dataloader[0]), dim=1)
        correct = (predictions == val_set.dataloader[1]).float()
        accuracy = correct.mean()
    return accuracy

def train_acc():
    with torch.no_grad():
        predictions = torch.argmax(model(train_set.dataloader[0].cuda()), dim=1)
        correct = (predictions == train_set.dataloader[1].cuda()).float()
        accuracy = correct.mean()
    return accuracy

In [28]:
dataset = {}
train_inputs = []
train_labels = []
for i, data in enumerate(train_set.dataset):
    image, label = data
    train_inputs.append(torch.flatten(image, start_dim=1).long().cuda())
    train_labels.append(torch.tensor(label).cuda())

test_inputs = []
test_labels = []
for i, data in enumerate(val_set.dataset):
    image, label = data
    test_inputs.append(torch.flatten(image, start_dim=1).long().cuda())
    test_labels.append(torch.tensor(label).cuda())

dataset['train_input'] = torch.stack(train_inputs)
dataset['train_label'] = torch.stack(train_labels)
dataset['test_input'] = torch.stack(test_inputs)
dataset['test_label'] = torch.stack(test_labels)

In [51]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import cv2

# Set the path to your local image folder
image_folder = './data'

# Define a custom dataset class to load images from local folder
class LocalImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = []
        self.labels = []
        label_dict = {}
        for idx, label in enumerate(os.listdir(image_folder)):
            label_dict[label] = idx
            for file in os.listdir(os.path.join(image_folder, label)):
                if file.endswith('.jpg') or file.endswith('.png'):  # adjust file extensions as needed
                    self.images.append(os.path.join(image_folder, label, file))
                    self.labels.append(label_dict[label])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        label = self.labels[index]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))  # resize to fixed shape
        img = img[12:116, 12:116]  # center crop to fixed shape
        if self.transform:
            img = self.transform(img)
        return img, label

# Create dataset and data loader for training and testing
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = LocalImageDataset(os.path.join(image_folder, 'train'), transform=transform)
test_dataset = LocalImageDataset(os.path.join(image_folder, 'test'), transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

X_train = []
y_train = []
for img, label in train_loader:
    img = img.numpy().squeeze()  # convert tensor to numpy array
    for i in range(img.shape[0]):
        img_resized = cv2.resize(img[i], (128, 128))  # resize to fixed shape
        X_train.append(img_resized.astype(float))
        y_train.append(label[0])

X_train = np.stack(X_train, axis=0)
y_train = np.array(y_train)

# Calculate mean and std of training data
mean, std = np.mean(X_train), np.std(X_train)
print(f"{mean=}")
print(f"{std=}")

X_test = []
y_test = []
for img, label in test_loader:
    img = img.numpy().squeeze()  # convert tensor to numpy array
    for i in range(img.shape[0]):
        img_resized = cv2.resize(img[i], (128, 128))  # resize to fixed shape
        X_test.append(img_resized.astype(float))
        y_test.append(label[0])

X_test = np.stack(X_test, axis=0)
y_test = np.array(y_test)

# Normalize testing data using mean and std of training data
X_test = (X_test - mean) / std
X_train = (X_train - mean) / std

mean=0.5351909170230562
std=0.20247474194162862


In [73]:
dataset = {}

dataset["train_input"] = (
    torch.from_numpy(np.array([data[0] for data in train_set])).long().to(device)
)
dataset["train_label"] = torch.from_numpy(np.array([data[1] for data in train_set])).long().to(device)

dataset["test_input"] = (
    torch.from_numpy(np.array([data[0] for data in val_set])).long().to(device)
)
dataset["test_label"] = torch.from_numpy(np.array([data[1] for data in val_set])).long().to(device)

In [78]:
batch_size = 78
height = 128
width = 128
channels = 3

# Assuming your image batch is a tensor with shape (batch_size, channels, height, width)
image_batch = dataset["train_label"]

# Reshape the image batch to have shape (batch_size, features)
dataset["train_label"] = image_batch.view(batch_size, -1)

#dataset["train_input"].shape

In [79]:
dataset["train_input"]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [45]:
class CustomDataset:
    def __init__(self, custom_dataset):
        self.custom_dataset = custom_dataset

    def __getitem__(self, key):
        if key == 'train_input':
            return [img for img, _ in self.custom_dataset]
        elif key == 'test_input':
            return [label for _, label in self.custom_dataset]
        else:
            raise KeyError(f"Unknown key {key}")

    def __len__(self):
        return len(self.custom_dataset)
    
dataset = train_set.dataset
dataset = CustomDataset(dataset)

In [80]:
import time
from tqdm import tqdm

print("[INFO] Start training")
print("---------------------")
startTime = time.time()

#dataset['train_input'] = dataset['train_input'].unsqueeze(1)

# Train the model
results = model.train(
    dataset,
    opt="LBFGS",
    steps=EPOCHS,
    batch=16,
    loss_fn=torch.nn.CrossEntropyLoss(),
    metrics=(train_acc, test_acc)
)

[INFO] Start training
---------------------


description:   0%|                                                           | 0/25 [00:00<?, ?it/s]


RuntimeError: shape '[16, 16384]' is invalid for input of size 786432

In [ ]:
# from torch.optim import Adam
# from torch.cuda.amp import autocast, GradScaler

# # initialize the CatSpotter model
# print("[INFO] initializing the CNN model...")
# model = MyCNNModel(
#     # n_filters=N_FILTERS, 
#     kernel_size=KERNEL_SIZE
#     ).cuda()
# # initialize our optimizer and loss function
# opt = Adam(model.parameters(), lr=INIT_LR)
# criterion = nn.CrossEntropyLoss()
# # initialize a dictionary to store training history
# H = {
#     "train_loss": [],
#     "train_acc": [],
#     "val_loss": [],
#     "val_acc": []
# }
# # Create a GradScaler for mixed-precision training
# scaler = GradScaler()

[INFO] initializing the CNN model...


In [ ]:
# Initialize model checkpoint saving location
PATH = 'models/04-26-2024'

In [ ]:
# Save model checkpoint
def save_ckp(state, checkpoint_dir, epoch_nr):
    f_path = checkpoint_dir + f'/epoch_{epoch_nr}.pt'
    torch.save(state, f_path)

In [ ]:
import time
from tqdm import tqdm

print("[INFO] Start training")
print("---------------------")
startTime = time.time()

for epoch in tqdm(range(EPOCHS)):

    total_batch = len(train_set.dataset)//BATCH_SIZE
    
    # Training phase
    #model.train()  # Set the model to train mode

    # Get statistics
    epoch_loss = 0
    len_dataset = 0
    
    for step, (batch_images, batch_labels) in enumerate(train_set,0):
        X, Y = batch_images, batch_labels

        opt.zero_grad()

        #X = X.view(X.size(0),-1).cuda()
        pred = model(X.cuda())
        loss = criterion(pred, Y.cuda())

        loss.backward()
        opt.step()

        epoch_loss += pred.shape[0] * loss.item()
        len_dataset += pred.shape[0]
        if (step) % 10 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, EPOCHS, step+1, total_batch, loss.item()))

    epoch_loss = epoch_loss/ len_dataset
    print('Epoch: ', epoch + 1, '| train loss : %0.4f' % epoch_loss)

    # # Save the epoch model parameters
    # checkpoint = {
    #     'epoch': epoch + 1,
    #     'state_dict': model.state_dict(),
    #     'optimizer': opt.state_dict()
    # }

    # save_ckp(checkpoint, PATH , epoch)

    
    # Validation phase
    #model.eval()  # Set the model to evaluation mode
    with torch.inference_mode():  
        running_loss = 0
        for step, (images, labels) in enumerate(val_set):
            
            #x = images.view(images.size(0),-1).cuda()

            outputs = model(images.cuda())
            loss = criterion(outputs, labels.cuda())

            running_loss += loss.item()
            
    running_loss = running_loss / len(val_set.dataset)
    print('Epoch: ', epoch +1, '| val loss : %0.4f' % running_loss )

# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
    endTime - startTime))

[INFO] Start training
---------------------


  0%|          | 0/25 [06:08<?, ?it/s]


RuntimeError: bad allocation

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
    #model.eval()
    preds = []
    labels = []
    for batch in val_set:
        x, y = batch
        x = x.view(x.size(0),-1).cuda()
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())
        labels.extend(y.cpu().numpy())
        
print(preds, labels)

print(classification_report(labels, preds))

[INFO] evaluating network...
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.50      1.00      0.67         9

    accuracy                           0.50        18
   macro avg       0.25      0.50      0.33        18
weighted avg       0.25      0.50      0.33        18



c:\Users\rdede\Documents\GitHub\SnowDetection\.env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rdede\Documents\GitHub\SnowDetection\.env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rdede\Documents\GitHub\SnowDetection\.env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
    model.eval()
    preds = []
    labels = []
    for batch in train_set:
        x, y = batch
        x = x.to(device)
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())
        labels.extend(y.cpu().numpy())

print(classification_report(labels, preds))

[INFO] evaluating network...


NameError: name 'torch' is not defined